In [3]:
from dask_gateway import Gateway
from dask_gateway.auth import BasicAuth
auth = BasicAuth(password="pangeo_dask")
#auth="jupyterhub"
gateway= Gateway(
    "http://api-daskhub-dask-gateway.daskhub:8000/",
    auth=auth,
)

In [21]:
clusters=gateway.list_clusters()
for cluster in clusters:
    cluster = gateway.connect(cluster.name)
    cluster.shutdown()

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:1014: RuntimeWarning: coroutine 'rpc.close_rpc' was never awaited
  self.scheduler_comm.close_rpc()


In [23]:
gateway.list_clusters()

[]

In [24]:
cluster = gateway.new_cluster()
cluster.scale(16)

In [25]:
from distributed import Client
client=Client(cluster)
cluster

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:1274: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| lz4     | 4.0.0  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


2022-07-28 00:33:44,499 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [27]:
from distributed.diagnostics.plugin import PipInstall
extra_packages=["xarray","netCDF4","pooch","h5netcdf","pystac-client"]
plugin=PipInstall(extra_packages,restart=True)
client.register_worker_plugin(plugin)
client.get_versions(packages=extra_packages,check=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:4239: UserWarning: Mismatched versions found

+----------+----------------+----------------+----------------+
| Package  | client         | scheduler      | workers        |
+----------+----------------+----------------+----------------+
| h5netcdf | 1.0.0          | None           | 1.0.1          |
| lz4      | 4.0.0          | None           | None           |
| netCDF4  | 1.5.8          | None           | 1.6.0          |
| pandas   | 1.4.2          | None           | 1.4.3          |
| python   | 3.9.13.final.0 | 3.10.5.final.0 | 3.10.5.final.0 |
| xarray   | 2022.3.0       | None           | 2022.6.0       |
+----------+----------------+----------------+----------------+
  warnings.warn(msg["warning"])


{'scheduler': {'host': {'python': '3.10.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.5.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1',
   'toolz': '0.11.2',
   'numpy': '1.22.4',
   'pandas': None,
   'lz4': None,
   'xarray': None,
   'netCDF4': None,
   'pooch': None,
   'h5netcdf': None,
   'pystac-client': None}},
 'workers': {'tls://10.244.1.6:45685': {'host': {'python': '3.10.5.final.0',
    'python-bits': 64,
    'OS': 'Linux',
    'OS-release': '5.4.0-109-generic',
    'machine': 'x86_64',
    'processor': '',
    'byteorder': 'little',
    'LC_ALL': 'None',
    'LANG': 'C.UTF-8'},
   'packages': {'python': '3.10.5.final.0',
    'dask': '2022.6.0',
    'distributed': '2022.6.0',
    'msgpack': '1.0.4',
    

## Distributed geospatial

In this section, we'll learn how to use a cluster of machines to create a cloud-free composite. Because of the amount of data involved, we'll distribute the workload on a cluster of machines.


As a reminder, Dask works by using high-level APIs (like xarray) to build task graphs. These task graphs are then executed by a scheduler:

![](https://docs.dask.org/en/latest/_images/dask-overview.svg)

Ideally, we don't need to change our use of high-level APIs like xarray or Dask DataFrame at all. We just swap out a distributed scheduler / cluster for the single-machine scheduler.

That said, it's worth remembering that going distributed makes many things more challenging. You need to think more about how to start workers, communication, distributed filesystems, environments, .... If you don't *need* a cluster, then you should avoid it. But for large enough problems, it's nice to have the option.

## Create a cluster

This JupyterHub is configured with [Dask Gateway](https://gateway.dask.org/), a convenient way to create Dask clusters.

Since we created a Client connected to that cluster, any operations using Dask will use that cluster. 

We've been really happy with Dask Gateway as a centralized way for users to create Dask clusters. Most importantly, you don't have to know anything about the underlying resource manager (Kubernetes in this case) to create and customize clusters.

## Access Earth systems science data

We'll work with Daymet data, specifically [monthly frequency for North America](https://planetarycomputer.microsoft.com/dataset/daymet-monthly-na), which provides summaries of temperature, precipitation, and other variables.

In [28]:
import pystac_client

stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1/")
c = stac.get_collection("daymet-monthly-na")
c

<CollectionClient id=daymet-monthly-na>

The data is provided in Zarr format. We can access the URL to the root of the Zarr store from the assets.

In [29]:
asset = c.assets["zarr-https"]
asset

<Asset href=https://daymeteuwest.blob.core.windows.net/daymet-zarr/monthly/na.zarr>

Which can be loaded into xarray. Notice that we rely on the STAC metadata to tell use which additional keywords we should pass to xarray (`consolidated=True`, in this case).

In [30]:
import xarray as xr

ds = xr.open_zarr(asset.href, **asset.extra_fields["xarray:open_kwargs"])
ds

<xarray.Dataset>
Dimensions:                  (y: 8075, x: 7814, time: 492, nv: 2)
Coordinates:
    lat                      (y, x) float32 dask.array<chunksize=(2000, 2000), meta=np.ndarray>
    lon                      (y, x) float32 dask.array<chunksize=(2000, 2000), meta=np.ndarray>
  * time                     (time) datetime64[ns] 1980-01-16T12:00:00 ... 20...
  * x                        (x) float32 -4.56e+06 -4.559e+06 ... 3.253e+06
  * y                        (y) float32 4.984e+06 4.983e+06 ... -3.09e+06
Dimensions without coordinates: nv
Data variables:
    lambert_conformal_conic  int16 ...
    prcp                     (time, y, x) float32 dask.array<chunksize=(12, 2000, 2000), meta=np.ndarray>
    swe                      (time, y, x) float32 dask.array<chunksize=(12, 2000, 2000), meta=np.ndarray>
    time_bnds                (time, nv) datetime64[ns] dask.array<chunksize=(12, 2), meta=np.ndarray>
    tmax                     (time, y, x) float32 dask.array<chunksize=(12, 2000, 2000), meta=np.ndarray>
    tmin                     (time, y, x) float32 dask.array<chunksize=(12, 2000, 2000), meta=np.ndarray>
    vp                       (time, y, x) float32 dask.array<chunksize=(12, 2000, 2000), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.6
    NCO:               netCDF Operators version 4.9.3 (Homepage = http://nco....
    Version_data:      Daymet Data Version 4.0
    Version_software:  Daymet Software Version 4.0
    citation:          Please see http://daymet.ornl.gov/ for current Daymet ...
    references:        Please see http://daymet.ornl.gov/ for current informa...
    source:            Daymet Software Version 4.0
    start_year:        1980

This is a pretty large Dataset. We can access subsets of the data and aggregate it, for example by taking the mean over time.

Since we connected a client to our cluster earlier, the computation will happen on our Dask cluster.

In [20]:
# note: this pass with local cluster, but not with gw cluster.
x = ds["prcp"].sel(time=slice("1985", "1986")).mean(dim="time").compute()



In [33]:
import hvplot.xarray  # noqa
#x = ds["prcp"].sel(time=slice("1985", None)).mean(dim="time").compute()


x = ds["prcp"].sel(time=slice("1985", "1986")).mean(dim="time").compute()
x.hvplot.image(x="x", y="y", rasterize=True, width=900, height=500)

:DynamicMap   []
   :Image   [x,y]   (prcp)

Or we can aggregate over space to get a timeseries.

In [34]:
ts = ds["tmax"].mean(dim=("y", "x")).compute()
ts.plot(figsize=(12, 6));

KeyboardInterrupt: 

In [ ]:
ts.groupby("time.year").mean().plot();

Overall we see a (bumpy) increase in temperature since 1980. But not all spots in North America have warmed equally. We can get a sense for which areas have increased more than others, relative to some baseline (pre-1989).

In [ ]:
import dask

summer_months = [6, 7, 8]
summer = ds.tmax.where(ds.time.dt.month.isin(summer_months), drop=True)

early_period = slice(None, "1988-12-31")
late_period = slice("1988-01-01", "2018-12-31")

early, late = dask.compute(
    summer.sel(time=early_period).mean(dim="time"),
    summer.sel(time=late_period).mean(dim="time"),
)

In [ ]:
increase = (late - early).coarsen(y=8, x=8, boundary="trim").mean()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 12))

increase.plot(ax=ax, vmin=-6, vmax=6, cmap="RdBu")
ax.set_axis_off()

In [ ]:
client.restart();

## Access remote sensing data

Now we'll make a cloudless composite from a bunch of setallite imagery. This will put a bunch of the techniques you've learned to use

1. Searching the STAC API for relevant scenes
2. Loading the items into an xarray DataArray
3. Timeseries manipulation with xarray
4. Distributed computing with Dask

In [ ]:
import pystac_client

bbox = [-59.69, -34.74, -58.24, -33.65]
stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = stac.search(
    bbox=bbox,
    datetime="2018-01-01/2020-12-31",
    collections=["sentinel-2-l2a"],
    limit=500,  # fetch items in batches of 500
    query={"eo:cloud_cover": {"lt": 25}},
)

In [ ]:
%%time
items = list(search.get_all_items())
print(len(items))

In [ ]:
import planetary_computer

signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [ ]:
import stackstac
import rasterio.enums

ds = stackstac.stack(
    signed_items,
    assets=["B04", "B03", "B02"],
    epsg=32721,
    bounds_latlon=bbox,
    resolution=100,
    resampling=rasterio.enums.Resampling.bilinear,
).where(lambda x: x > 0)
ds

In [ ]:
monthly_rgb = ds.resample(time="MS").median(dim="time")
monthly_rgb

In [ ]:
rgb = monthly_rgb.compute()

In [ ]:
rgb.plot.imshow(row="time", rgb="band", robust=True, size=6);